In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import StratifiedShuffleSplit

/Users/danielg/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
submission_sample = pd.read_csv("data/sample_submission.csv")
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
# create id - does this actually get used - should really be able to use VisitNumber
train['id'] = train.index.values
test['id'] = test.index.values

In [4]:
# could create variable with number of items per visit???
# will probably be quite useful??
    # will be able to create other variables like: departmenttype flat, etc - so quite a lot of other variables

unique_trip_items_train = train.groupby(['VisitNumber']).agg({'id': len})
unique_trip_items_train.columns = ['unique_trip_items']

unique_trip_items_test = test.groupby(['VisitNumber']).agg({'id': len})
unique_trip_items_test.columns = ['unique_trip_items']

# also total number by the scancounts...
trip_scancount_total_train = train.groupby(['VisitNumber']).agg({'ScanCount': sum})
trip_scancount_total_train.columns = ['ScanCount_total']

trip_scancount_total_test = test.groupby(['VisitNumber']).agg({'ScanCount': sum})
trip_scancount_total_test.columns = ['ScanCount_total']

In [5]:
train = train.merge(unique_trip_items_train, left_on='VisitNumber', right_index=True)
train = train.merge(trip_scancount_total_train, left_on="VisitNumber", right_index=True)
test = test.merge(unique_trip_items_test, left_on="VisitNumber", right_index=True)
test = test.merge(trip_scancount_total_test, left_on="VisitNumber", right_index=True)

In [6]:
#
# missing value imputation
#

# will set missing FinelineNumber to -1
train.FinelineNumber = train.FinelineNumber.fillna(-1)
test.FinelineNumber = test.FinelineNumber.fillna(-1)
# will set missing Upc to 0
train.Upc = train.Upc.fillna(0)
test.Upc = test.Upc.fillna(0)
# will set missing DepartmentDescription to Missing
train.DepartmentDescription = train.DepartmentDescription.fillna("missing")
test.DepartmentDescription = test.DepartmentDescription.fillna("missing")

In [7]:
train_grouped = train.groupby("VisitNumber", as_index=False).agg({'Weekday': lambda x: x.unique(),
                               'id': len,
                              'ScanCount': sum,
                             'TripType': lambda x: x.unique()})

test_grouped = test.groupby("VisitNumber", as_index=False).agg({'Weekday': lambda x: x.unique(),
                               'id': len,
                              'ScanCount': sum})

In [8]:
train_grouped = train_grouped.set_index("VisitNumber")
X_test = test_grouped.set_index("VisitNumber")

In [9]:
X_train = train_grouped.drop("TripType",axis=1)
y_train = train_grouped.TripType

In [10]:
#
# Day of week feature
#

days = X_train.Weekday.unique()
for day in days:
    X_train[str(day)] = (X_train.Weekday == str(day)).astype(int)
X_train = X_train.drop("Weekday",axis=1)

days = X_test.Weekday.unique()
for day in days:
    X_test[str(day)] = (X_test.Weekday == str(day)).astype(int)
X_test = X_test.drop("Weekday",axis=1)

#### department features

In [11]:
department_crosstab = pd.crosstab(train.VisitNumber, train.DepartmentDescription)
# sort index because the order of the columns needs to be identical in test and train
X_train = X_train.merge(department_crosstab.sort_index(axis=1), left_index=True, right_index=True)
department_crosstab_test = pd.crosstab(test.VisitNumber, test.DepartmentDescription)
# how to get missing columns that are IN the training but NOT test
# 
missing_col_test = department_crosstab.columns[~department_crosstab.columns.isin(department_crosstab_test.columns)].values[0]
# need to add the columns
department_crosstab_test[missing_col_test] = 0
X_test = X_test.merge(department_crosstab_test.sort_index(axis=1), left_index=True, right_index=True)
# need to make sure that all of the columns match
all(X_train.columns == X_test.columns)

True

#### fineline features

In [12]:
train.head(2)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,id,unique_trip_items,ScanCount_total
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000,0,1,-1
1,30,7,Friday,60538815980,1,SHOES,8931,1,2,2


In [13]:
# integrate fineline data

# want to do the same for FineLinenumber
fineline_train_crosstab = pd.crosstab(train.VisitNumber, train.FinelineNumber)
fineline_test_crosstab = pd.crosstab(test.VisitNumber, test.FinelineNumber)

In [14]:
fineline_not_in_train = fineline_test_crosstab.columns[~fineline_test_crosstab.columns.isin(fineline_train_crosstab.columns)].values
fineline_test_crosstab = fineline_test_crosstab.drop(fineline_not_in_train,axis=1)
# want to drop train columns that are not contained in test
# need a 'not in test' variable
fineline_not_in_test = fineline_train_crosstab.columns[~fineline_train_crosstab.columns.isin(fineline_test_crosstab.columns)].values
fineline_train_crosstab = fineline_train_crosstab.drop(fineline_not_in_test,axis=1)
# need to make sure that all of these match
all(fineline_test_crosstab.columns == fineline_train_crosstab.columns)

True

In [16]:
# switching this over to test sums - to emphasise this...??

# have a column mismatch need to fix this ....‚
fineline_ordered_sums = fineline_train_crosstab.sum(0).sort_values(ascending=False).iloc[0:100]
finelines_incl = fineline_ordered_sums.index
# .iloc[0:1000]

In [17]:
fineline_train_crosstab_reduced = fineline_train_crosstab.loc[:,finelines_incl]
fineline_test_crosstab_reduced = fineline_test_crosstab.loc[:,finelines_incl]

In [18]:
# merge into fineline one hot encoded into X_train and X_test
X_train = X_train.merge(fineline_train_crosstab_reduced, left_index=True, right_index=True)
X_test = X_test.merge(fineline_test_crosstab_reduced, left_index=True, right_index=True)

#### upc feature

In [ ]:
upc_train_crosstab = pd.crosstab(train.VisitNumber, train.Upc)
upc_test_crosstab = pd.crosstab(test.VisitNumber, test.Upc)

In [29]:
train.Upc.value_counts().iloc[0:100]

4011           7657
0              4129
60538862097    2501
7874235186     2067
7874235187     1883
4046           1652
68113107862    1515
60538871457    1487
3338320027     1405
4087           1370
60538871461    1343
4900000044     1018
4062            973
4065            953
4900003165      896
3338365020      856
7874235188      828
4900005010      815
68113163351     778
60538896309     778
4078            750
69922162117     745
7874211433      719
4093            698
4900000977      697
20966500000     686
60538819035     644
7874235201      622
4051            620
7225003706      618
               ... 
68113189617     378
4900002890      376
61126999100     372
4000042431      370
7874298393      365
1200080994      364
7225003712      363
9518801128      360
4900001278      360
4900005015      357
68113102889     357
7874203952      356
4693            345
3338311000      343
3338360002      343
4900001916      338
8265750406      332
30521500700     328
4029            326


#### check columns in test and train

Need to match or the model will not work

In [19]:
assert all(X_train.columns == X_test.columns)

#### create final csv

In [20]:
y_train = pd.DataFrame({"TripType":y_train})

In [21]:
X_train.to_csv("data/model_inputs/X_train.csv")

In [22]:
X_test.to_csv("data/model_inputs/X_test.csv")

In [23]:
y_train.to_csv("data/model_inputs/y_train.csv")

In [ ]:
train.head(1)